In [44]:
import sys
sys.path.append('../')
import simulation_tools as sim

import copy
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from cmdstanpy import CmdStanModel

# Remove cmdstanpy logs
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

The below function takes in a dataset of experiments done so far and returns the ratio that minimizes the posterior variance of $\mu_\theta$.

In [45]:
def optimize_next_ratio(model, expt_df, grid_ratio, num_subjects, reps):
    best_variance = float("inf")
    best_ratio = None

    for ratio in tqdm(grid_ratio, desc="Ratio", leave=False):
        variances = []

        for _ in tqdm(range(reps), desc="Repetition", leave=False):

            # Do one more experiment
            params = sim.expt_df_to_params(expt_df)
            params["num_subjects_per_expt"] = np.array([num_subjects])
            params["prop_treatment"] = np.array([ratio])
            new_expt_df = sim.simulate_experiments(params)

            # Append to expt_df
            full_expt_df = pd.concat([expt_df, new_expt_df], ignore_index=True)

            # Observe posterior summary
            summary = sim.posterior_summary(model, full_expt_df)
            variances.append(summary["sigma_theta"])

        avg_variance = np.mean(np.array(variances))
        if avg_variance < best_variance:
            best_variance = avg_variance
            best_ratio = ratio

    return best_ratio

In [46]:
def adaptive_sim_experiments(model, params, grid_ratios, reps):
    params = copy.deepcopy(params)
    best_ratios = [0.5]

    num_subjects_per_expt = params["num_subjects_per_expt"]
    num_expts = len(num_subjects_per_expt)

    # Generate initial experiment
    params["num_subjects_per_expt"] = np.array([params["num_subjects_per_expt"][0]])
    params["prop_treatment"] = np.array([0.5])
    expt_df = sim.simulate_experiments(params)

    for expt_id in tqdm(range(1, num_expts), desc="Experiment"):

        num_subjects = num_subjects_per_expt[expt_id]
        best_ratio = optimize_next_ratio(model, expt_df, grid_ratios, num_subjects, reps)
        best_ratios.append(best_ratio)

        # Generate another experiment
        params["num_subjects_per_expt"] = np.array([num_subjects])
        params["prop_treatment"] = np.array([best_ratio])

        # Append to existing expt_df
        new_expt_df = sim.simulate_experiments(params)
        expt_df = pd.concat([expt_df, new_expt_df], ignore_index=True)

    return {
        "expt_df": expt_df,
        "ratios": best_ratios,
    }
    

In [47]:
num_expts = 38
chick_params = {
    "num_subjects_per_expt": np.full(num_expts, sim.CHICK_NUM_SUBJECTS_PER_EXPT),
    "prop_treatment": np.full(num_expts, 0.5),
    "mu_b": 0,
    "mu_theta": sim.CHICK_MU_THETA,
    "sigma_b": sim.CHICK_SIGMA_B,
    "sigma_theta": sim.CHICK_SIGMA_THETA,
    "sigma_treatment": sim.CHICK_SIGMA_TREATMENT,
    "sigma_control": sim.CHICK_SIGMA_CONTROL,
}
chick_data = sim.simulate_experiments(chick_params)
model = CmdStanModel(stan_file="../basic_mlm.stan")
grid_ratios = np.linspace(0.1, 0.9, 9)

In [48]:
res = adaptive_sim_experiments(model, chick_params, grid_ratios, 30)

Experiment:   0%|          | 0/37 [00:00<?, ?it/s]

Ratio:   0%|          | 0/9 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

...

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

In [49]:
print(res["ratios"])

[0.5, np.float64(0.30000000000000004), np.float64(0.4), np.float64(0.6), np.float64(0.8), np.float64(0.30000000000000004), np.float64(0.6), np.float64(0.7000000000000001), np.float64(0.6), np.float64(0.1), np.float64(0.6), np.float64(0.1), np.float64(0.5), np.float64(0.5), np.float64(0.1), np.float64(0.5), np.float64(0.1), np.float64(0.1), np.float64(0.4), np.float64(0.6), np.float64(0.1), np.float64(0.2), np.float64(0.2), np.float64(0.1), np.float64(0.30000000000000004), np.float64(0.1), np.float64(0.2), np.float64(0.5), np.float64(0.1), np.float64(0.1), np.float64(0.2), np.float64(0.1), np.float64(0.30000000000000004), np.float64(0.1), np.float64(0.2), np.float64(0.4), np.float64(0.1), np.float64(0.1)]


In [50]:
optimized_expts = res["expt_df"]
balanced_expts = sim.simulate_experiments(chick_params)

In [51]:
opt_posterior = sim.get_bayes_estimates(optimized_expts, model=model, oracle=True)
balanced_posterior = sim.get_bayes_estimates(balanced_expts, model=model, oracle=True)

In [52]:
print(f"Optimized posterior sample MSE: {(opt_posterior["sample_error"] ** 2).mean()}")
print(f"Balanced posterior sample MSE: {(balanced_posterior["sample_error"] ** 2).mean()}")

Optimized posterior sample MSE: 0.0022804683531694537
Balanced posterior sample MSE: 0.0010281778742659796


Extreme case: what if $s_{0j} = 0$ for all $j$?

In [54]:
params_sigma_0j_0 = {
    "num_subjects_per_expt": np.full(num_expts, sim.CHICK_NUM_SUBJECTS_PER_EXPT),
    "mu_b": 0,
    "mu_theta": sim.CHICK_MU_THETA,
    "sigma_b": sim.CHICK_SIGMA_B,
    "sigma_theta": sim.CHICK_SIGMA_THETA,
    "sigma_treatment": 2 * sim.CHICK_SIGMA_TREATMENT,
    "sigma_control": sim.CHICK_SIGMA_TREATMENT,
}

In [ ]:
balanced_expts = sim.simulate_experiments(chick_params)
opt_posterior2 = sim.get_bayes_estimates(res2["expt_df"], model=model, oracle=True)
balanced_posterior2 = sim.get_bayes_estimates(balanced_expts, model=model, oracle=True)

In [56]:
params_sigma_0j_0 = {
    "num_subjects_per_expt": [np.full(num_expts, sim.CHICK_NUM_SUBJECTS_PER_EXPT)],
    "prop_treatment": [np.full(num_expts, 0.5), np.concatenate(
    (np.full(5, 0.5), np.linspace(0.5, 0.95, sim.CHICK_NUM_EXPTS - 5))
)],
    "mu_b": [0],
    "mu_theta": [sim.CHICK_MU_THETA],
    "sigma_b": [sim.CHICK_SIGMA_B],
    "sigma_theta": [sim.CHICK_SIGMA_THETA],
    "sigma_treatment": [2 * sim.CHICK_SIGMA_TREATMENT],
    "sigma_control": [sim.CHICK_SIGMA_TREATMENT],
}

In [57]:
df = sim.evaluate_params_means(model, 20, params_sigma_0j_0)

Parameter Combination:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/20 [00:00<?, ?it/s]

Repetition:   0%|          | 0/20 [00:00<?, ?it/s]

In [73]:
prop_const = np.full(38, 0.5)
prop_varying = np.concatenate(
    (np.full(5, 0.5), np.linspace(0.5, 0.95, sim.CHICK_NUM_EXPTS - 5))
)
params_varying_sigma_treatment = {
    "num_subjects_per_expt": [np.full(num_expts, sim.CHICK_NUM_SUBJECTS_PER_EXPT)],
    "prop_treatment": [np.full(num_expts, 0.5), np.concatenate(
    (np.full(5, 0.5), np.linspace(0.5, 0.95, sim.CHICK_NUM_EXPTS - 5))
)],
    "mu_b": [0],
    "mu_theta": [sim.CHICK_MU_THETA],
    "sigma_b": [sim.CHICK_SIGMA_B],
    "sigma_theta": [sim.CHICK_SIGMA_THETA],
    "sigma_treatment": np.linspace(1, 2, 20) * sim.CHICK_SIGMA_TREATMENT,
    "sigma_control": [sim.CHICK_SIGMA_TREATMENT],
}

In [74]:
df = sim.evaluate_params_means(model, 30, params_varying_sigma_treatment)

Parameter Combination:   0%|          | 0/40 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

...

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

Repetition:   0%|          | 0/30 [00:00<?, ?it/s]

In [75]:
df_bayes = df[df["estimator"] == "bayes"]
const_prop_df = df_bayes[df_bayes["prop_treatment"].apply(lambda x: np.array_equal(x, prop_const))]
varying_prop_df = df_bayes[df_bayes["prop_treatment"].apply(lambda x: np.array_equal(x, prop_varying))]

In [82]:
import plotly.express as px

# Calculate the ratio
ratio = np.sqrt(varying_prop_df["sample_mse"].values / const_prop_df["sample_mse"].values)

# Create the plot
fig = px.line(
    x=varying_prop_df["sigma_treatment"] / sim.CHICK_SIGMA_TREATMENT,
    y=ratio,
    labels={"x": "Sigma Treatment / Sigma Control", "y": "MSE varying p / MSE constant p"},
    title="MSE Ratio vs Sigma Ratio for Posterior Estimator"
)

fig.show()